# 必要なライブラリのインストール

In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install janome
!pip install scikit-learn
!pip install pandas
!pip list

Package         Version   
--------------- ----------
autopep8        1.5       
beautifulsoup4  4.8.2     
certifi         2019.11.28
chardet         3.0.4     
idna            2.9       
Janome          0.3.10    
joblib          0.14.1    
numpy           1.18.1    
pandas          1.0.1     
pip             20.0.2    
pycodestyle     2.5.0     
python-dateutil 2.8.1     
pytz            2019.3    
requests        2.23.0    
scikit-learn    0.22.2    
scipy           1.4.1     
setuptools      40.8.0    
six             1.14.0    
soupsieve       2.0       
urllib3         1.25.8    


In [2]:
import sys
# 自分の環境では venv内のパッケージにパスが通っていなかったため、パスを追加
venv_packages_path = '/Users/kawahara.masahiro/org/devio/ml-tfidf/venv/lib/python3.7/site-packages' #edit yourself
if venv_packages_path not in sys.path:
    sys.path.append(venv_packages_path)

# URLリストから各ブログのテキスト情報を取得する

## URLリスト取得
- 対象のURLリストは `./contents/url_list.txt` に格納

In [3]:
url_list_path = "./contents/url_list.txt" #edit yourself
url_list = []
with open(url_list_path) as f:
    url_list = [l.replace('\n', '') for l in f.readlines()]
# 確認
print("len(url_list): {}".format(len(url_list)))
print("url_list[0]: {}".format(url_list[0]))

len(url_list): 48
url_list[0]: https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/


## マッピング作成
- インデックス <---> 各情報(URL, テキスト, TF-IDF値など)

In [4]:
BLOG = {}
for i, url in enumerate(url_list):
    BLOG[i] = {}
    BLOG[i]["url"] = url
# 確認
print("len(BLOG): {}".format(len(BLOG)))
print("BLOG[0][\"url\"]: {}".format(BLOG[0]["url"]))

len(BLOG): 48
BLOG[0]["url"]: https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/


## ブログテキストのリストを取得
- 各URLリストから `./util/get_blog_texts.py` を使ってブログテキストを取得する

In [5]:
from util.get_blog_texts import get_blog_texts

In [6]:
import time 

for i in BLOG.keys():
    url = BLOG[i]["url"]
    print("#{} getting texts from: {}".format(i, url))
    BLOG[i]["texts"] = get_blog_texts(url)
    time.sleep(1)

#0 getting texts from: https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
#1 getting texts from: https://dev.classmethod.jp/cloud/aws/aws-cisco-site-to-site-vpn/
#2 getting texts from: https://dev.classmethod.jp/cloud/aws/higobashi-aws-11-deepracer/
#3 getting texts from: https://dev.classmethod.jp/cloud/aws/aws-summit-osaka-2019-297-input-output/
#4 getting texts from: https://dev.classmethod.jp/cloud/aws/aws-cdk-intro-nw/
#5 getting texts from: https://dev.classmethod.jp/etc/summit-osaka-2019-299-basic/
#6 getting texts from: https://dev.classmethod.jp/cloud/aws/vscode-botostubs-boto3/
#7 getting texts from: https://dev.classmethod.jp/etc/site_vpn_azure_aws/
#8 getting texts from: https://dev.classmethod.jp/cloud/aws/awsspec-by-jupyter/
#9 getting texts from: https://dev.classmethod.jp/cloud/aws/deepracer-new-input-parameters/
#10 getting texts from: https://dev.classmethod.jp/cloud/aws/aws-summit-osaka-2019-298-security/
#11 getting texts from: https://dev.clas

In [7]:
# 確認
print("len(BLOG[0][texts]): {}".format(len(BLOG[0]["texts"])))
print("len(BLOG[0][texts][0]): {}".format(BLOG[0]["texts"][0]))

len(BLOG[0][texts]): 65
len(BLOG[0][texts][0]): Application Load Balancer (ALB), Network Load Balancer (NLB) はターゲットにローカルIPアドレス を指定できます。VPCピアリング接続先のインスタンスや、Direct Connect・VPN接続先のオンプレのサーバーをターゲットグループに登録することができます。


## テキスト前処理
- 今回は日本語テキストを想定しているので、空白を削除します
- アルファベットは全て小文字化します

In [8]:
for i in BLOG.keys():
    BLOG[i]["texts"] = [t.replace(' ','').lower() for t in BLOG[i]["texts"]]

In [9]:
# 確認
print("len(BLOG[0][texts]): {}".format(len(BLOG[0]["texts"])))
print("len(BLOG[0][texts][0]): {}".format(BLOG[0]["texts"][0]))

len(BLOG[0][texts]): 65
len(BLOG[0][texts][0]): applicationloadbalancer(alb),networkloadbalancer(nlb)はターゲットにローカルipアドレスを指定できます。vpcピアリング接続先のインスタンスや、directconnect・vpn接続先のオンプレのサーバーをターゲットグループに登録することができます。


# janome で形態素解析
- 各テキストの単語を抽出
- 分かち書きした結果を `BLOG[i]["wakati"]` に格納
- 参考
    - [Janome documentation](https://mocobeta.github.io/janome/)
    - [Janome API #POSStopFilter](https://mocobeta.github.io/janome/api/janome.html#janome.tokenfilter.POSStopFilter)
    - [Janome API #Token](https://mocobeta.github.io/janome/api/janome.html#janome.tokenizer.Token)

In [10]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter

In [11]:
tokenizer = Tokenizer()
token_filters = [POSStopFilter(['記号','助詞','助動詞','動詞'])]
a = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

In [12]:
# 確認(1文を解析)
test_tokens = a.analyze(BLOG[0]["texts"][0])
for t in test_tokens:
    print(t)

applicationloadbalancer	名詞,固有名詞,組織,*,*,*,applicationloadbalancer,*,*
(	名詞,サ変接続,*,*,*,*,(,*,*
alb	名詞,一般,*,*,*,*,alb,*,*
),	名詞,サ変接続,*,*,*,*,),,*,*
networkloadbalancer	名詞,一般,*,*,*,*,networkloadbalancer,*,*
(	名詞,サ変接続,*,*,*,*,(,*,*
nlb	名詞,一般,*,*,*,*,nlb,*,*
)	名詞,サ変接続,*,*,*,*,),*,*
ターゲット	名詞,一般,*,*,*,*,ターゲット,ターゲット,ターゲット
ローカル	名詞,一般,*,*,*,*,ローカル,ローカル,ローカル
ip	名詞,一般,*,*,*,*,ip,*,*
アドレス	名詞,一般,*,*,*,*,アドレス,アドレス,アドレス
指定	名詞,サ変接続,*,*,*,*,指定,シテイ,シテイ
vpc	名詞,一般,*,*,*,*,vpc,*,*
ピアリング	名詞,一般,*,*,*,*,ピアリング,*,*
接続	名詞,サ変接続,*,*,*,*,接続,セツゾク,セツゾク
先	名詞,接尾,一般,*,*,*,先,サキ,サキ
インスタンス	名詞,一般,*,*,*,*,インスタンス,インスタンス,インスタンス
directconnect	名詞,固有名詞,組織,*,*,*,directconnect,*,*
vpn	名詞,固有名詞,組織,*,*,*,vpn,*,*
接続	名詞,サ変接続,*,*,*,*,接続,セツゾク,セツゾク
先	名詞,接尾,一般,*,*,*,先,サキ,サキ
オンプレ	名詞,一般,*,*,*,*,オンプレ,*,*
サーバー	名詞,一般,*,*,*,*,サーバー,サーバー,サーバー
ターゲット	名詞,一般,*,*,*,*,ターゲット,ターゲット,ターゲット
グループ	名詞,一般,*,*,*,*,グループ,グループ,グループ
登録	名詞,サ変接続,*,*,*,*,登録,トウロク,トーロク
こと	名詞,非自立,一般,*,*,*,こと,コト,コト


In [13]:
# 解析
for i in BLOG.keys():
    texts_flat = "。".join(BLOG[i]["texts"])
    tokens = a.analyze(texts_flat)
    BLOG[i]["wakati"] = ' '.join([t.surface for t in tokens])
# 確認
print("BLOG[0][wakati]: {}".format(BLOG[0]["wakati"]))

BLOG[0][wakati]: applicationloadbalancer ( alb ), networkloadbalancer ( nlb ) ターゲット ローカル ip アドレス 指定 vpc ピアリング 接続 先 インスタンス directconnect vpn 接続 先 オンプレ サーバー ターゲット グループ 登録 こと 今回 実際 オンプレ サーバー ターゲット 登録 alb リバースプロキシ 環境 構築 構築 検証 環境 以下 通り 私 上 検証 環境 構築 経緯 説明 ( スキップ 次 章 : 環境 構築 ok )。 directconnect ( dx ) 先 外部 サービス 通信 制限 接続 元 vpc cidr 指定 値 イケ ない ケース この 要件 ため 前々回 ブログ 前回 ブログ 自前 プロキシサーバー 構築 が 通信 要件 次第 aws マネージドサービス 構成 こと ゴール 既存 環境 ( vpc ) app サーバー 外部 サービス ローカル ip http ( s ) 通信 こと 新規 vpc 作成 その 上 applicationloadbalancer ( alb ) リバースプロキシ 環境 構築 alb ターゲット 外部 サービス ローカル ip 指定 検証 環境 再掲 構築 cloudformation ( cfn ) こちら 作成 cfn テンプレート vpn 接続 周り 以前 ブログ どおり そちら 参照 自宅 aws vpn 接続 ( cisco ルータ 編 )。。 alb cfn テンプレート 中身 説明 下記 3 リソース 作成 ▼— loadbalancer ---。▼— targetgroup ---。▼— listener ---。。 まず ターゲット ステータス 登録 ターゲット ( オンプレサーバー ) ステータス healthy app サーバー ログイン オンプレサーバー サービス 利用 検証 ( オンプレサーバー web サービス ( http ) )。 app サーバー ログイン alb アクセス アクセス 検証 内容 味気 オンプレサーバー アクセス 別名 ( エイリアス ) 作成 route 53 新規 ホスト ゾーン 作成 vpc プライベートホストゾーン レコード 作成 エイ

# CountVectorizer で単語出現頻度を計算
- `WORDS` に名詞一覧を格納
- `BLOG[i]["bow"]` に BoW を格納
- 参考
    - [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
import random
vectorizer = CountVectorizer()

In [15]:
X = vectorizer.fit_transform([BLOG[i]["wakati"] for i in BLOG.keys()])

In [18]:
WORDS = vectorizer.get_feature_names()
# 確認
print(WORDS[:100])
print(WORDS[-100:])

['00', '000', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '107', '11', '12', '1234567890', '14', '149', '15', '16', '160', '168', '17', '172', '18', '184', '19', '192', '197', '1977', '20', '2001', '2002', '201', '2015', '2016', '2018', '2019', '2020', '20200204', '21', '2114', '22', '224', '24', '240030', '25', '254', '256', '26', '27', '30', '3128', '320', '34', '35', '3600', '375', '40', '403', '406', '41', '44', '463', '50', '500', '53', '57', '60', '618', '65', '66', '74', '746', '80', '830', '841', '950', '96', '99', '999999999', '__', 'aaaa', 'abcdef', 'abooleanflagtoindicatewhethertheagenthascrashedintoanotherobject', 'abooleanflagtoindicatewhethertheagenthasofftrack', 'about', 'aboutexecutionpolicies', 'abouttheevent', 'abs', 'accelerated', 'acceleratedsite', 'accelerator', 'accelerators', 'accessdenied', 'accessyourawsregionsfasterusingtheawsmanagementconsole', 'account', 'acl', 'acllocalnetsrcxxx']
['豊富', '負債', '負荷', '責任', '責務', '費用', '資料', '資料集', '質問'

In [19]:
for i, bow in enumerate(X.toarray()):
    BLOG[i]["bow"] = bow

In [20]:
# 確認 (1ブログの高頻出の単語をリスト)
n = 6
print("url of BLOG[0]: {}".format(BLOG[0]["url"]))
print("list high frequency(>={}) words of BLOG[0]".format(n))
for w_idx, count in enumerate(BLOG[0]["bow"]):
    if count >= n:
        print("  - {}\t{}: WORDS[{}]".format(count, WORDS[w_idx], w_idx))

url of BLOG[0]: https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
list high frequency(>=6) words of BLOG[0]
  - 13	alb: WORDS[112]
  - 7	ip: WORDS[603]
  - 8	オンプレサーバー: WORDS[1481]
  - 7	サーバー: WORDS[1585]
  - 6	サービス: WORDS[1586]
  - 11	ターゲット: WORDS[1665]
  - 12	作成: WORDS[2046]
  - 6	指定: WORDS[2365]
  - 6	接続: WORDS[2369]
  - 6	検証: WORDS[2471]
  - 10	構築: WORDS[2481]
  - 9	環境: WORDS[2551]


# TF値の導出関数を作ってみる
## 参考
- [【技術解説】単語の重要度を測る？TF-IDFとOkapi BM25の計算方法とは](https://mieruca-ai.com/ai/tf-idf_okapi-bm25/)

In [21]:
def calc_tf(b_idx, w_idx):
    """b_idx 番目のブログの WORD[w_idx] の TF値を算出する"""
    # WORD[w_idx] の出現回数の和
    word_count = BLOG[b_idx]["bow"][w_idx]
    if word_count == 0:
        return 0.0
    # 全単語の出現回数の和
    sum_of_words = sum(BLOG[b_idx]["bow"])
    # TF値計計算
    return word_count/float(sum_of_words)

## 1blogの TF値を計算してみる

In [22]:
# 確認
index = 0
print("# TF values of blog:{}".format(BLOG[index]["url"]))
sample_tfs = [calc_tf(index, w_idx) for w_idx, word in enumerate(WORDS)]
tfs_sorted = sorted(enumerate(sample_tfs), key=lambda x:x[1], reverse=True)
for i, tf in tfs_sorted[:20]:
    print("{}\t{}".format(WORDS[i], round(tf, 4)))

# TF values of blog:https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
alb	0.0405
作成	0.0374
ターゲット	0.0343
構築	0.0312
環境	0.028
オンプレサーバー	0.0249
ip	0.0218
サーバー	0.0218
サービス	0.0187
指定	0.0187
接続	0.0187
検証	0.0187
app	0.0156
http	0.0156
vpc	0.0156
こと	0.0156
ローカル	0.0156
通信	0.0156
vpn	0.0125
yaml	0.0125


# IDFの導出関数を作ってみる

In [23]:
import math

def calc_idf(w_idx):
    """WORD[w_idx] の IDF値を算出する"""
    # 総文書数
    N = len(BLOG.keys())
    # 単語 word が出現する文書数 df を計算
    df = len([i for i in BLOG.keys() if BLOG[i]["bow"][w_idx] > 0])
    # idf を計算
    return math.log2(N/float(df + 1))

## 全単語のIDFを計算してみる

In [24]:
IDFS = [calc_idf(w_idx) for w_idx, word in enumerate(WORDS)]

In [25]:
# 確認
idfs_sorted  = sorted(enumerate(IDFS), key=lambda x:x[1], reverse=True)
print("# IDF values")
for w_idx, idf in idfs_sorted[:20]:
    print("{}\t{}".format(WORDS[w_idx], round(idf, 4)))
print("︙")
for w_idx, idf in idfs_sorted[-20:]:
    print("{}\t{}".format(WORDS[w_idx], round(idf, 4)))

# IDF values
000	4.585
04	4.585
05	4.585
07	4.585
08	4.585
09	4.585
107	4.585
1234567890	4.585
149	4.585
160	4.585
172	4.585
184	4.585
197	4.585
1977	4.585
2001	4.585
2002	4.585
201	4.585
2015	4.585
2018	4.585
2020	4.585
︙
参考	1.1255
機能	1.1255
記事	1.1255
必要	1.0614
情報	0.9411
確認	0.9411
使用	0.8845
利用	0.8301
場合	0.8301
実行	0.8301
設定	0.8301
今回	0.727
環境	0.727
この	0.5406
以下	0.5406
ため	0.4975
よう	0.4975
作成	0.4975
aws	0.415
こと	0.2274


# TF-IDF を計算する

In [26]:
def calc_tfidf(b_idx, w_idx):
    """b_idx 番目のブログの WORD[w_idx] の TF-IDF値を算出する"""
    return calc_tf(b_idx, w_idx) * calc_idf(w_idx)

In [27]:
# 確認
index = 0
print("# TF-IDF values of blog:{}".format(BLOG[index]["url"]))
sample_tfidfs = [calc_tfidf(index, w_idx) for w_idx, word in enumerate(WORDS)]
tfidfs_sorted = sorted(enumerate(sample_tfidfs), key=lambda x:x[1], reverse=True)
for i, tfidf in tfidfs_sorted[:20]:
    print("{}\t{}".format(WORDS[i], round(tfidf, 4)))

# TF-IDF values of blog:https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
alb	0.1215
オンプレサーバー	0.1143
ターゲット	0.1028
サーバー	0.0564
http	0.0558
app	0.0433
構築	0.0417
外部	0.0407
検証	0.0397
ip	0.0388
エイリアス	0.0374
リバースプロキシ	0.0374
ローカル	0.0352
接続	0.0352
yaml	0.0346
通信	0.0312
80	0.0286
登録	0.0282
awscloudformation	0.028
vpn	0.0265


## 全ブログの TF, TF-IDF を計算する
- 全ブログの TF, TF-IDF を計算して、それぞれ上位10個をテーブル表記する

In [28]:
for b_idx in BLOG.keys():
    BLOG[b_idx]["tf"] = [calc_tf(b_idx, w_idx) for w_idx, w in enumerate(WORDS)]
    BLOG[b_idx]["tfidf"] = [calc_tfidf(b_idx, w_idx) for w_idx, w in enumerate(WORDS)]

In [29]:
import pandas as pd
n = 5
for b_idx in BLOG.keys():
    print("\n## TOP {} TF, TF-IDF of blog: {}".format(n, BLOG[b_idx]["url"]))
    tfs_sorted = sorted(enumerate(BLOG[b_idx]["tf"]), key=lambda x:x[1], reverse=True)
    tfidfs_sorted = sorted(enumerate(BLOG[b_idx]["tfidf"]), key=lambda x:x[1], reverse=True)
    # display
    buffer = [
        [i+1, WORDS[tfs_sorted[i][0]], WORDS[tfidfs_sorted[i][0]]]
        for i in range(0,n)
    ]
    print(pd.DataFrame(buffer, columns=["", "TF上位", "TF-IDF上位"]).to_markdown(showindex=False))


## TOP 5 TF, TF-IDF of blog: https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
|    | TF上位     | TF-IDF上位       |
|---:|:-----------|:-----------------|
|  1 | alb        | alb              |
|  2 | 作成       | オンプレサーバー |
|  3 | ターゲット | ターゲット       |
|  4 | 構築       | サーバー         |
|  5 | 環境       | http             |

## TOP 5 TF, TF-IDF of blog: https://dev.classmethod.jp/cloud/aws/aws-cisco-site-to-site-vpn/
|    | TF上位   | TF-IDF上位   |
|---:|:---------|:-------------|
|  1 | aws      | active       |
|  2 | vpn      | 自宅         |
|  3 | 自宅     | vpn          |
|  4 | 接続     | cisco        |
|  5 | active   | 接続         |

## TOP 5 TF, TF-IDF of blog: https://dev.classmethod.jp/cloud/aws/higobashi-aws-11-deepracer/
|    | TF上位       | TF-IDF上位   |
|---:|:-------------|:-------------|
|  1 | aws          | 料金         |
|  2 | 料金         | usd          |
|  3 | awsdeepracer | awsdeepracer |
|  4 | usd          | cd           |
|  5 | 時間         | ci           